In [ ]:
from pathlib import Path
import pydicom
import numpy as np

In [ ]:
#Code is created to handle both sagital and transversal scans.

#Define crop size for each scan size.
crop_sag320 = [[0, 20], #sag_x
               [70, 280], #sag_z
               [150, 285]] #sag_y
crop_sag384 = [[0, 20], #sag_x
               [70, 280], #sag_z
               [170, 305]] #sag_y
crop_sag576 = [[0, 20], #sag_x
               [120, 330], #sag_z
               [185, 320]] #sag_y


crop_tra448 = [[8, 62-8], #tra_z
               [205, 367], #tra_y
               [156, 297]] #tra_x

crop_tra528 = [[8, 62-8], #tra_z
               [255, 417], #tra_y
               [156+30, 297+30]] #tra_x

crop_tra528_68 = [[8+5, 62-3], #tra_z
               [255, 417], #tra_y
               [156+30, 297+30]] #tra_x

crop_tra528_70 = [[8, 62-8], #tra_z
               [255, 417], #tra_y
               [156+30, 297+30]] #tra_x

crop_tra528_72 = [[8, 62-8], #tra_z
               [255, 417], #tra_y
               [156+30, 297+30]] #tra_x

crop_tra528_74 = [[8+12, 62+4], #tra_z
               [255, 417], #tra_y
               [156+30, 297+30]] #tra_x

def crop_vol(volume, crop_values):
    volume_new = volume[
        crop_values[0][0]:crop_values[0][1],
        crop_values[1][0]:crop_values[1][1], #Højde
        crop_values[2][0]:crop_values[2][1]] #Bredde
    return volume_new, crop_values

In [ ]:
data_folder = Path('/tf/data')
folders = ['/tf/data/endo',
           '/tf/data/prost',
           '/tf/data/recti']

In [ ]:
#Look through all folders
n = 0
for folder in folders:
    for case in Path(folder).glob('8*'): #all Test cases starts with 8
        for scan in case.iterdir():
            dicom_files = [file for file in scan.iterdir()]
            slice_location_map = {}
            
            #Find slice location for all scans
            for file in dicom_files:
                dicom = pydicom.dcmread(file)
                slice_location_map[file] = dicom.SliceLocation
            
            #Sort scans by slicelocation
            dicom_files.sort(key=lambda x: slice_location_map[x])
    
            #Define volume
            volume = []
            for dicom in dicom_files:
                image = pydicom.dcmread(dicom)
                volume.append(image.pixel_array)
            volume = np.array(volume)
    
            #Crop if sagital scan and print original and cropped slices.
            if 'SAG' in str(scan):
                if volume.shape == (20,320,320):
                    volume_new, crop_values = crop_vol(volume, crop_sag320)
            
                elif volume.shape == (20,384,384):
                    volume_new, crop_values = crop_vol(volume, crop_sag384)

                elif volume.shape == (20, 576, 576):
                    volume_new, crop_values = crop_vol(volume, crop_sag576)
    
                #Stop iteration if scan is not accounted for in if statements
                else:
                    print(volume.shape, scan)
                    raise ValueError()
    
            #Crop if transversal scan and print original and cropped slices.
            if 'TRA' in str(scan) or 'TSE' in str(scan):
                if volume.shape == (62,448,448) or volume.shape == (65,448,448):
                    volume_new, crop_values = crop_vol(volume, crop_tra448)
                                 
                elif volume.shape == (66,528,528):
                    volume_new, crop_values = crop_vol(volume, crop_tra528)
                
                elif volume.shape == (68,528,528):
                    volume_new, crop_values = crop_vol(volume, crop_tra528_68)
                    
                elif volume.shape == (70,528,528):
                    volume_new, crop_values = crop_vol(volume, crop_tra528_70)
                    
                elif volume.shape == (72,528,528):
                    volume_new, crop_values = crop_vol(volume, crop_tra528_72)
    
                elif volume.shape == (74,528,528) or volume.shape == (75,528,528) or volume.shape == (78,528,528):
                    volume_new, crop_values = crop_vol(volume, crop_tra528_74)
    
                #Stop iteration if scan is not accounted for in if statements
                else:
                    print(volume.shape, scan)
                    raise ValueError()
    
            scan_folder = scan
            #Stop iteration if cropped volume is wrong size
            if volume_new.shape == (46, 162, 141) or volume_new.shape == (20, 210, 135):
                save_path = data_folder / 'cropped' / 'test data' / scan_folder.relative_to(data_folder)
                save_path.mkdir(parents=True, exist_ok=True)
                for i, slice in enumerate(volume_new):
                    save_file_path = save_path / f'{i:02d}_{dicom_files[i+crop_values[0][0]].stem}'
                    np.save(save_file_path, slice)
            else:
                print(case)
                print(volume.shape)
                print(volume_new.shape)
                raise ValueError()
        n += 1
        print(f'Saved case {n}: {scan}')